# <h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Torronto</font></h1>

#### 1. Import numpy and pandas

In [84]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

#### 2. Read Wikipedia
Check the number of table

In [85]:
URL="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df_wiki=pd.read_html(URL)
len(df_wiki)

3

check the dataframe

In [86]:
df_wiki[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### 3. Drop all the Not assigned Borough

In [126]:
df_tor=df_wiki[0][df_wiki[0]['Borough']!='Not assigned'].reset_index(drop=True)
df_tor.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### 4. Check if there is somme not assigned Neighbourhood

In [88]:
df_tor[df_tor['Neighbourhood']=='Not assigned']

,index,Postal Code,Borough,Neighbourhood


#### 5. Check the shape of the dataframe

In [89]:
df_tor.shape

(103, 4)

#### 6. define a fonction to call latitude and longitude

In [90]:
!wget https://cocl.us/Geospatial_data

--2021-02-06 14:05:48--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.194, 169.63.96.176
Connecting to cocl.us (cocl.us)|169.63.96.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-02-06 14:05:49--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-02-06 14:05:49--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]


In [91]:
geo_data= pd.read_csv('Geospatial_data')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [104]:
def get_coord (postal_code):
    
    coordo=geo_data[geo_data['Postal Code']==postal_code]
    coordo.reset_index(inplace=True)
    latitude = coordo.at[0,'Latitude']
    longitude = coordo.at[0,'Longitude']
    #print('The geograpical coordinate of {} is {}, {}.'.format(postal_code,latitude, longitude))
    return latitude, longitude

print('geocoder function defined!')

geocoder function defined!


In [105]:
lat,long=get_coord('M1G')
print(lat)

43.7709921


#### 7. Loop inside the dataframe and append the lat and long columns

In [127]:
coord=pd.DataFrame(columns=['latitude','longitude'])
coord.drop(coord.index,inplace=True)
for pc in df_tor['Postal Code']:
    latitude, longitude = get_coord(pc)
    coord_pc=pd.DataFrame([[latitude,longitude]],columns=['latitude','longitude'])
    coord=coord.append(coord_pc)
   
coord.shape
coord.reset_index(inplace=True,drop=True)

In [138]:
df_coord=pd.concat([df_tor, coord], axis=1)
df_coord.head()

,Postal Code,Borough,Neighbourhood,latitude,longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
